In [85]:
import numpy
import pandas
import matplotlib.pyplot as pyplot
import seaborn
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from joblib import dump
import pickle
from sklearn.cluster import KMeans,DBSCAN
from joblib import dump

In [56]:
word_lemitizer=WordNetLemmatizer()
Regular_expression_definition_for_html_tags=re.compile('<.*?>')
Regular_expression_definition_for_digits=re.compile('\d+\s|\s\d+|\s\d+\s')
english_stop_words=stopwords.words('english')
def preprocessing_of_sentence(text):
    word_to_be_handled=[
    "not",
    "no",
    "very"
    ]
    text=Regular_expression_definition_for_html_tags.sub(r" ",text)
    text=Regular_expression_definition_for_digits.sub(r" ",text)
    punctuations = [".",",","!","?","'",'"',":",";","*","-","/","+","%","$","#","@","(",")","[","]","{","}"]
    for i in punctuations:
        text = text.replace(i," ")
    text=text.lower().split()
    text=[word for word in text if word not in english_stop_words and len(word)>1 or word in word_to_be_handled]
    text=[word_lemitizer.lemmatize(word) for word in text]
    return text

In [57]:
class_column='Score'
review_column='Text'

In [58]:
class_list=["Very Bad","Bad","Average","Good","Awesome"]

In [59]:
filename="Reviews.csv"

In [60]:
Total_Data=pandas.read_csv('../Data/Processed_Data/'+filename)

In [61]:
Total_Data[class_column]=Total_Data[class_column]-1

In [62]:
Total_Data

Text  Score
0      using hour year work great take every day saw ...      4
1      cosco month sold searched found amazon thank w...      4
2      huge lavender fan never tried edible drinkable...      4
3      always fan kashi golean crunch find perfect ba...      4
4      year old mastiff despite tough appearance bit ...      4
...                                                  ...    ...
24995  since willing try alot new thing thought would...      0
24996  cat picky eater no interest eating pill pocket...      0
24997  shocked received shipment rip instant coffee e...      0
24998  looked forward delicious hot cocoa very disapp...      0
24999  product hydrogenated vegetable oil first ingre...      0

[25000 rows x 2 columns]

## Shuffelling and Spliting in features and labels

In [63]:
Total_Data=Total_Data.sample(frac=1)

In [64]:
total_Features=Total_Data[review_column]
total_Labels=Total_Data[class_column]

# Vecorization

In [65]:
Vectorizer=TfidfVectorizer()

In [66]:
def vectorize_the_input(data):
    return Vectorizer.transform(data)

In [67]:
Total_Vector_Data=Vectorizer.fit_transform(total_Features.values.astype('U'))

In [68]:
Total_Vector_Data.shape

(25000, 26522)

In [84]:
with open("../Models/Tockenizer.pickle","wb") as f:
    pickle.dump(Vectorizer,f)

In [70]:
training_size=int(len(Total_Data)*0.8)

In [71]:
Training_Vector_Data=Total_Vector_Data[:training_size]
Testing_Vector_Data=Total_Vector_Data[training_size:]

In [72]:
training_Labels=Total_Data[class_column][:training_size]
testing_Labels=Total_Data[class_column][training_size:]

## Input Preparation

In [91]:
input_="i have found the very bad restaurent"
input_=preprocessing_of_sentence(input_)
print(input_)
input_=vectorize_the_input([" ".join(input_)])

['found', 'very', 'bad', 'restaurent']


# Unsupervised Method

## Clustering

In [74]:
kmeans = KMeans(n_clusters=5, random_state=0).fit(Total_Vector_Data)

In [75]:
kmeans

KMeans(n_clusters=5, random_state=0)

In [76]:
kmeans.cluster_centers_

array([[7.55905380e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [7.10120470e-04, 3.41696514e-05, 8.67535155e-06, ...,
        3.23603060e-05, 0.00000000e+00, 0.00000000e+00],
       [2.50047881e-04, 1.16818071e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.30458068e-04, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 1.46434927e-04, 0.00000000e+00],
       [5.69092561e-04, 1.36882022e-04, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 6.34831700e-05]])

In [77]:
class_list[kmeans.predict(input_)[0]]

'Bad'

In [94]:
predictions=kmeans.predict(Testing_Vector_Data)

In [95]:
kmeans.predict(input_)

array([1])

In [83]:
# dump(kmeans, '../Models/Model.joblib')

['../Models/Model.joblib']

## DBSCAN

In [87]:
dbscan=DBSCAN(min_samples=5000)

In [88]:
dbscan.fit(Total_Vector_Data)

DBSCAN(min_samples=5000)

In [93]:
dbscan.fit_predict(input_)

array([-1], dtype=int64)